# Obligations in District 11

Using data from the Division of Local Assistance [Obligation list](https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated)

In [3]:
#! pip install cpi

In [4]:
import numpy as np
import pandas as pd
from siuba import *

import altair as alt
import altair_saver
from plotnine import *

from IPython.display import Markdown

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from calitp import to_snakecase
import intake

import _clean_data
import _dla_utils

In [5]:
pd.options.display.float_format = '{:,.2f}'.format

In [6]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("dla_df.parquet")

In [7]:
#parameters cell
subset = 11

In [9]:
df = (df>>filter(_.dist==subset))

In [84]:
df.sample(10)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
17823,Obligated,STPBIPL,6066(170),San Diego Association Of Governments,2020-12-01,2020-12-03,2020-12-23,2020-12-23,2020-12-29,"17,941,000.00","-17,941,000.00",0.00,Authorized,6066,11.00,E-76 approved on,38.00,20.00,6.00,V11,East Otay Mesa Port Of Entry From The Border W...,Construct Port Of Entry (tc),2.00,2020-10-26,2020-12-03,SDAG,6066,"2,020.00",San Diego Association of Governments,0.00,"18,783,861.12","-18,783,861.12"
12085,Obligated,HSIPL,5211(029),El Cajon,2014-04-17,2014-04-18,2014-04-22,2014-04-23,2014-04-23,"107,900.00",0.00,"120,000.00",Authorized,nan,11.00,E-76 approved on E-76 approved on,1.00,5.00,0.00,CAL105 / CAL306,Various Locations Throughout The City,Upgrade Signs,2.00,NaT,NaT,SJCOG,5211,"2,014.00",El Cajon,"137,352.91","123,503.16",0.00
18472,Obligated,HSIPL,5207(045),La Mesa,2021-02-04,2021-02-04,2021-02-04,2021-02-04,2021-02-08,0.00,0.00,0.00,Authorized,5207,11.00,E-76 approved on,7.00,0.00,4.00,CAL105,Various Intersections Within The City Of La Mesa.,"Upgrade Regulatory, Warnings, And School Signs...",2.00,2021-01-28,2021-01-29,SDAG,5207,"2,021.00",La Mesa,0.00,0.00,0.00
14294,Obligated,CML,6066(121),San Diego Association Of Governments,2019-05-23,2019-05-24,2019-06-06,2019-06-06,2019-06-12,"-1,300,000.00",0.00,"-1,300,000.00",Authorized,6066,11.00,E-76 approved on,1.00,13.00,6.00,SAN201,Along Palm Street West Of Pacific Highway,Pedestrian Improvements (tc),10.00,2019-05-23,2019-05-23,SDAG,6066,"2,019.00",San Diego Association of Governments,"-1,377,863.58","-1,377,863.58",0.00
4827,Obligated,STPL,5174(028),Holtville,2017-02-07,2017-03-07,2017-03-07,2017-03-07,2017-03-13,"583,999.00",0.00,"659,663.00",Authorized,5174,11.00,E-76 approved on Prepared on 2/14/2017,28.00,0.00,6.00,IMP,Sixth Street Pavement Improvements Between Hol...,Pavement Improvement,1.00,NaT,NaT,SCAG,5174,"2,017.00",Holtville,"729,231.25","645,587.70",0.00
1480,Obligated,HSIPL,5382(029),Vista,2018-05-23,2018-05-23,2018-05-23,2018-05-23,2018-06-01,-0.30,0.00,-0.30,Authorized,5382,11.00,E-76 approved on,NaN,0.00,9.00,CAL105,Sr78 Off Ramp At Sycamore Dr. Vista,Widen The Off Ramp,6.00,NaT,NaT,SDAG,5382,"2,018.00",Vista,-0.32,-0.32,0.00
8030,Obligated,SRTSL,5429(022),Santee,2015-10-21,2015-11-03,2015-11-09,2015-11-09,2015-11-15,"381,000.00",0.00,"439,004.00",Authorized,5429,11.00,E-76 approved on Prepared on 10/6/2015,13.00,6.00,6.00,CAL 104 / 275,"In Santee At Mast Blvd. / Jeremy St., Hill Cre...","Construct Raised Median, Ped Island, Curb Ramps",2.00,NaT,NaT,SDAG,5429,"2,015.00",Santee,"501,891.91","435,578.76",0.00
17100,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,0.12,0.00,0.12,Prog Code Q230,7503,11.00,FTA transferred on 8/3/2020,NaN,28.00,3.00,nan,None,FTA Transfer,1.00,NaT,NaT,NON-MPO,7503,"2,020.00",San Diego Metropolitan Transit System,0.13,0.13,0.00
4608,Obligated,HSIPL,5081(019),Escondido,2017-03-30,2017-04-03,2017-04-04,2017-04-04,2017-04-12,"627,344.00",0.00,"627,344.00",Authorized,5081,11.00,E-76 approved on Prepared on 2/14/2017,4.00,1.00,8.00,CAL105/CAL347,El Norte Pkwy/fig & East Valley Pkwy/date,Install Two New Traffic Signals (tc),2.00,NaT,NaT,SDAG,5081,"2,017.00",Escondido,"693,503.88","693,503.88",0.00
14245,Obligated,FERPL,5958(104),Imperial County,2019-05-06,2019-05-06,2019-05-06,2019-05-06,2019-05-17,"547,366.00",0.00,"547,366.00",Authorized,5958,11.00,E-76 approved on,11.00,0.00,11.00,nan,Imperial Highway From I-8 To Sr 98 In Ocotillo...,Access Improvement Include A 2-inch Overlay Of...,1.

In [85]:
len(df)

947

In [11]:
df_years = _dla_utils.count_all_years(df)

In [12]:
df_years

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_mpo,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
10,2022,11.00,"-97,032.29","12,278,702.11","10,824,544.92","-7,464.02","944,515.55","832,657.30",3.00,9.00,9.00,12.00,12.00,12.00
9,2021,11.00,"-22,048,981.00","62,736,743.15","39,484,060.96","-206,065.24","586,324.70","369,009.92",3.00,25.00,22.00,81.00,83.00,81.00
8,2020,11.00,"5,286,649.00","114,150,595.24","144,299,163.08","40,981.78","884,888.34","1,118,598.16",3.00,27.00,23.00,102.00,109.00,92.00
7,2019,11.00,"-19,971,304.00","62,159,902.64","69,392,790.82","-183,222.97","570,274.34","636,631.11",2.00,22.00,22.00,90.00,90.00,80.00
0,2018,11.00,"39,547,944.00","56,736,895.52","109,554,198.54","380,268.69","545,547.07","1,053,405.76",2.00,29.00,21.00,81.00,85.00,84.00
1,2017,11.00,"79,668,325.00","52,728,400.99","153,790,118.86","781,062.01","516,945.11","1,507,746.26",3.00,22.00,25.00,90.00,91.00,88.00
2,2016,11.00,"165,393,693.00","103,380,124.70","359,905,282.95","1,355,686.01","847,378.07","2,950,043.30",3.00,29.00,23.00,101.00,105.00,98.00
3,2015,11.00,"45,445,388.00","46,553,372.18","196,077,466.16","363,563.10","372,426.98","1,568,619.73",6.00,28.00,21.00,100.00,102.00,96.00
4,2014,11.00,0.00,"47,694,110.84","61,749,532.44",0.00,"381,552.89","493,996.26",10.00,29.00,25.00,102.00,109.00,97.00
6,2013,11.00,0.00,"3,890,130.70","4,317,169.63",0.00,"432,236.74","479,685.51",4.00,7.00,8.00,9.00,9.00,9.00


In [13]:
df_top = _dla_utils.find_top(df)

In [14]:
df_top

,value,count,variable
0,HSIPL,275,prefix
1,CML,101,prefix
2,STPL,98,prefix
3,BRLS,45,prefix
4,BPMPL,37,prefix
...,...,...,...
16,Calipatria,23,primary_agency_name
17,Vista,21,primary_agency_name
18,Carlsbad,14,primary_agency_name
19,Oceanside,14,primary_agency_name


## Which Agencies have Obligations

### How many Unique Agencies

In [15]:
_dla_utils.calculate_data_all(df, 'primary_agency_name', aggfunc="nunique")

,dist,primary_agency_name
0,11.00,31


### Agencies with the most Obligations

In [17]:
df_top>>filter(_.variable == 'primary_agency_name')

,value,count,variable
0,San Diego County,102,primary_agency_name
1,San Diego Association of Governments,101,primary_agency_name
2,Imperial County,83,primary_agency_name
3,San Diego,76,primary_agency_name
4,Chula Vista,65,primary_agency_name
5,National City,44,primary_agency_name
6,San Marcos,41,primary_agency_name
7,El Cajon,40,primary_agency_name
8,Brawley,35,primary_agency_name
9,Encinitas,35,primary_agency_name


### Agencies with the least Obligations

In [18]:
(df>>count(_.primary_agency_name)>>arrange(-_.n)).tail(10)

,primary_agency_name,n
12,Escondido,12
30,Westmorland,12
25,San Diego Metropolitan Transit System,8
16,Imperial County Transportation Commission,7
8,Del Mar,6
18,Lemon Grove,3
7,Coronado,2
26,San Diego Unified Port District,2
6,Chula Vista Elementary School District,1
21,Poway,1


## Transit Funding

In [102]:
transit = df[df['type_of_work'].str.contains(
              'transit|bus|train',
              case=False, na=False)]

In [103]:
print(f"Out of {len(df)} obligations, {len(transit)} are transit related")

Out of 947 obligations, 1 are transit related


In [104]:
transit>>count(_.primary_agency_name)>>arrange(-_.n)

,primary_agency_name,n
0,San Diego Metropolitan Transit System,1


In [105]:
#transit>>group_by(_.prefix)>>count(_.type_of_work)>>arrange(-_.n)

In [106]:
transit>>count(_.type_of_work)>>arrange(-_.n)

,type_of_work,n
0,Low-floor Light Rail Transit Vehicles,1


In [107]:
transit>>count(_.prefix)>>arrange(-_.n)

,prefix,n
0,FTASTPL,1


In [108]:
transit_agencies = df[df['primary_agency_name'].str.contains(
              'Transit|Rail',
              case=False, na=False)]

In [109]:
transit_agencies>>count(_.primary_agency_name)

,primary_agency_name,n
0,San Diego Metropolitan Transit System,8


In [110]:
transit_agencies>>count(_.type_of_work)>>arrange(-_.n)
#using this to determine how many FTA transfers there are  

,type_of_work,n
0,FTA Transfer,7
1,Low-floor Light Rail Transit Vehicles,1


In [111]:
_dla_utils.calculate_data_all(transit_agencies, 'total_requested', aggregate_by='primary_agency_name', aggfunc="mean")

,primary_agency_name,total_requested
0,San Diego Metropolitan Transit System,"2,450,000.00"


In [112]:
transit = transit.append(transit_agencies, ignore_index=True)

/tmp/ipykernel_1000/3814245088.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [114]:
transit.drop_duplicates()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
0,FHWA,FTASTPL,7503(003),Sd Metro Trns,2022-01-25,2022-01-25,2022-02-08,2022-02-14,NaT,"10,000,000.00",0.00,"10,000,000.00",prog code Y230,7503,11.00,FTA transferred waiting at FHWA,NaN,14.00,6.00,nan,FTA transfer,Low-floor Light Rail Transit Vehicles,1.00,NaT,NaT,NON-MPO,7503,"2,022.00",San Diego Metropolitan Transit System,"9,637,984.26","9,637,984.26",0.00
1,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,"9,342,958.12",0.00,"9,342,958.12",Prog Code Z230,7503,11.00,FTA transferred on 8/3/2020,NaN,28.00,3.00,nan,None,FTA Transfer,1.00,NaT,NaT,NON-MPO,7503,"2,020.00",San Diego Metropolitan Transit System,"9,781,886.62","9,781,886.62",0.00
2,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,0.65,0.00,0.65,Prog Code H230,7503,11.00,FTA transferred on 8/3/2020,NaN,28.00,3.00,nan,None,FTA Transfer,1.00,NaT,NaT,NON-MPO,7503,"2,020.00",San Diego Metropolitan Transit System,0.68,0.68,0.00
3,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,"1,312.48",0.00,"1,312.48",Prog Code L230,7503,11.00,FTA transferred on 8/3/2020,NaN,28.00,3.00,nan,None,FTA Transfer,1.00,NaT,NaT,NON-MPO,7503,"2,020.00",San Diego Metropolitan Transit System,"1,374.14","1,374.14",0.00
4,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,0.51,0.00,0.51,Prog Code L23E,7503,11.00,FTA transferred on 8/3/2020,NaN,28.00,3.00,nan,None,FTA Transfer,1.00,NaT,NaT,NON-MPO,7503,"2,020.00",San Diego Metropolitan Transit System,0.53,0.53,0.00
5,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,"12,354.63",0.00,"12,354.63",Prog Code L23R,7503,11.00,FTA transferred on 8/3/2020,NaN,28.00,3.00,nan,None,FTA Transfer,1.00,NaT,NaT,NON-MPO,7503,"2,020.00",San Diego Metropolitan Transit System,"12,935.05","12,935.05",0.00
6,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,0.12,0.00,0.12,Prog Code Q230,7503,11.00,FTA transferred on 8/3/2020,NaN,28.00,3.00,nan,None,FTA Transfer,1.00,NaT,NaT,NON-MPO,7503,"2,020.00",San Diego Metropolitan Transit System,0.13,0.13,0.00
7,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,"243,373.49",0.00,"243,373.49",Prog Code 33C0,7503,11.00,FTA transferred on 8/3/2020,NaN,28.00,3.00,nan,None,FTA Transfer,1.00,NaT,NaT,NON-MPO,7503,"2,020.00",San Diego Metropolitan Transit System,"254,807.08","254,807.08",0.00


### Charting

In [37]:
_dla_utils.basic_bar_chart((df_top>>filter(_.variable=='primary_agency_name')), 'value', 'count', 'value', subset, 'Agencies With The Most Obligations')

alt.Chart(...)

In [38]:
df_top>>filter(_.variable=='prefix')

,value,count,variable
0,HSIPL,275,prefix
1,CML,101,prefix
2,STPL,98,prefix
3,BRLS,45,prefix
4,BPMPL,37,prefix
5,ER,33,prefix
6,FERPL,27,prefix
7,RPSTPL,24,prefix
8,ATPL,23,prefix
9,SRTSL,20,prefix


In [39]:
_dla_utils.basic_bar_chart((df_top>>filter(_.variable=='prefix')), 'value', 'count', 'value', subset, 'Most used Prefix Codes')

alt.Chart(...)

In [40]:
(df>>group_by(_.primary_agency_name)>>count(_.prefix)>>arrange(-_.n))

,primary_agency_name,prefix,n
33,Chula Vista,HSIPL,38
39,El Cajon,HSIPL,37
98,National City,HSIPL,31
91,La Mesa,HSIPL,28
157,San Diego County,HSIPL,25
...,...,...,...
156,San Diego County,HPLU,1
162,San Marcos,BRLKS,1
173,Vista,TCSPDE,1
175,Westmorland,HPLU,1


## Number of Unique Prefix Codes

In [148]:
(_dla_utils.get_nunique(df, 'prefix', 'primary_agency_name'))

,primary_agency_name,n
23,San Diego Association of Governments,20
15,Imperial County,19
22,San Diego,19
24,San Diego County,13
3,Caltrans,11
11,Encinitas,8
5,Chula Vista,7
14,Imperial,6
19,National City,6
27,San Marcos,6


In [149]:
(_dla_utils.get_nunique(df, 'primary_agency_name', 'prefix'))

,prefix,n
43,HSIPL,16
60,STPL,14
19,CML,12
31,FERPL,9
37,HPLU,8
...,...,...
61,STPLER,1
62,STPLN,1
63,STPLX,1
65,TCSP,1


In [43]:
_dla_utils.basic_bar_chart(((_dla_utils.get_nunique(df, 'prefix', 'primary_agency_name')).head(30)),
                            'primary_agency_name', 'n', 'primary_agency_name', subset, 'Agencies With The Most Unique Prefix Codes')

alt.Chart(...)

In [151]:
df_top>>filter(_.variable=="prefix")

,value,count,variable
0,HSIPL,275,prefix
1,CML,101,prefix
2,STPL,98,prefix
3,BRLS,45,prefix
4,BPMPL,37,prefix
5,ER,33,prefix
6,FERPL,27,prefix
7,RPSTPL,24,prefix
8,ATPL,23,prefix
9,SRTSL,20,prefix


## Most Common Types of Work by Agnecy

In [45]:
_dla_utils.get_nunique(df, 'primary_agency_name', 'type_of_work')

,type_of_work,n
29,Bridge Replacement,4
31,Bridge Replacement (tc),3
104,Emergency Opening,3
115,FTA Transfer,3
310,Road Rehabilitation,3
...,...,...
402,Widen Road From 4 To 6 Lanes With Raised Media...,1
403,"Widen Roadway To Add Bike Lane, Sidewalks, Sho...",1
404,Widen The Off Ramp,1
405,Work Occur In 2 Road Segments; 1st Segment Is ...,1


In [46]:
_dla_utils.calculate_data_head(df, "primary_agency_name",'type_of_work', aggfunc="count")

,type_of_work,primary_agency_name
24,Bridge Preventive Maintenance,30
115,FTA Transfer,18
104,Emergency Opening,17
377,Upgrade Traffic Signals (interconnect),14
29,Bridge Replacement,12
247,Pedestrian Improvements (tc),10
246,Pedestrian Bridge,9
31,Bridge Replacement (tc),9
258,Pr,8
299,Replace 2-lane Bridge With 4-lane Bridge,7


In [47]:
df>>group_by(_.primary_agency_name)>>count(_.type_of_work)>>arrange(-_.n)>>filter(_.n>=8)

,primary_agency_name,type_of_work,n
351,San Diego County,Bridge Preventive Maintenance,19
267,San Diego,Bridge Preventive Maintenance,11
361,San Diego County,Emergency Opening,11
328,San Diego Association of Governments,FTA Transfer,10
338,San Diego Association of Governments,Pedestrian Improvements (tc),10
136,Encinitas,Pr,8


## Most Common Project Locations

In [48]:
df_top>>filter(_.variable=='project_location')

,value,count,variable
0,Various Locations In San Diego County,14,project_location
1,Along Palm Street West Of Pacific Highway,10,project_location
2,Various Locations Throughout The City,9,project_location
3,San Diego County,8,project_location
4,North Brown From East Alamo To Delta Street,7,project_location
5,"Via Vera Cruz, Brdg No. 57c0780",7,project_location
6,"Willow St @ Sweetwater River, Bridge 57c0011",7,project_location
7,Winter Gardens Boulevard @ Winter Gardens Drive,7,project_location
8,Heritage Road Bridge From Main St / Nirvana Av...,6,project_location
9,South Palm Avenue,6,project_location


## Number of Obligations by Year

In [49]:
df_top>>filter(_.variable=='prepared_y')

,value,count,variable
0,"2,020.00",129,prepared_y
1,"2,014.00",125,prepared_y
2,"2,015.00",125,prepared_y
3,"2,016.00",122,prepared_y
4,"2,019.00",109,prepared_y
5,"2,021.00",107,prepared_y
6,"2,018.00",104,prepared_y
7,"2,017.00",102,prepared_y
8,"2,022.00",13,prepared_y
9,"2,013.00",9,prepared_y


In [50]:
df>>group_by(_.primary_agency_name)>>count(_.prepared_date)>>arrange(-_.n)

,primary_agency_name,prepared_date,n
731,San Diego County,2020-09-28,7
749,San Diego Metropolitan Transit System,2020-07-03,7
712,San Diego County,2019-02-20,4
143,Chula Vista,2019-08-01,3
254,Encinitas,2017-08-10,3
...,...,...,...
845,Westmorland,2018-04-23,1
846,Westmorland,2018-04-30,1
847,Westmorland,2018-05-01,1
848,Westmorland,2019-01-07,1


### Chart

In [51]:
_dla_utils.basic_line_chart((df_top>>filter(_.variable=='prepared_y')), 'value', 'count', subset, 'Obligations by Year')

alt.Chart(...)

## Agencies with the most unique project numbers

In [52]:
df>>group_by(_.primary_agency_name, _.prefix)>>count(_.project_no)>>arrange(-_.n)>>filter(_.n>5)

,primary_agency_name,prefix,project_no,n
288,San Diego Association of Governments,CML,6066(121),10
27,Calipatria,CMSTPL,5243(015),7
58,Chula Vista,BHLS,5203(016),7
59,Chula Vista,BRLS,5203(039),7
252,San Diego,BPMPL,5004(188),7
391,San Diego Metropolitan Transit System,FTASTPL,7503(001),7
395,San Marcos,BRLS,5381(033),7
14,Brawley,STPL,5167(037),6
208,La Mesa,HSIPL,5207(038),6


## Funding Amounts

In [96]:
_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean")

,primary_agency_name,total_requested
3,Caltrans,"12,536,254.82"
16,Imperial County Transportation Commission,"3,656,763.71"
23,San Diego Association of Governments,"3,006,584.09"
22,San Diego,"2,548,911.28"
25,San Diego Metropolitan Transit System,"2,450,000.00"
27,San Marcos,"1,426,124.38"
1,Calexico,"1,040,094.85"
12,Escondido,"782,249.23"
5,Chula Vista,"637,313.89"
7,Coronado,"607,190.16"


In [98]:
_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="sum")

,primary_agency_name,total_requested
3,Caltrans,"363,551,389.87"
23,San Diego Association of Governments,"303,664,993.07"
22,San Diego,"193,717,257.56"
27,San Marcos,"58,471,099.51"
5,Chula Vista,"41,425,402.78"
16,Imperial County Transportation Commission,"25,597,346.00"
24,San Diego County,"25,505,084.26"
25,San Diego Metropolitan Transit System,"19,600,000.00"
19,National City,"13,856,568.86"
15,Imperial County,"13,821,379.25"


In [54]:
_dla_utils.calculate_data_all(df, 'total_requested', 'type_of_work', aggfunc="mean")

,type_of_work,total_requested
354,To Replace The San Elijo Lagoon Bridge And Uc....,"123,061,297.00"
125,Hov Lane (tc),"41,350,599.33"
34,Build 7.1 Miles Of Class L Bicycle Trail,"30,193,000.00"
30,Bridge Replacement (57c-0023),"27,693,945.60"
395,Widen Border Crossing Bridge Over The All Amer...,"10,791,721.50"
...,...,...
282,Reconstruction Of Parking Lot & Passenger Loading,"3,260.71"
139,Install Curb Ramps And Sidewalk,"2,139.36"
332,Safety Improvements,773.73
52,Conduct Environmental Study For Railroad/ Loca...,772.82


In [55]:
pd.options.display.float_format = '{:,.2f}'.format

In [100]:
df.describe()

,fed_requested,ac_requested,total_requested,dist,dist_processing_days,hq_processing_days,fhwa_processing_days,seq,prepared_y,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
count,947.00,947.00,947.00,947.00,864.00,935.00,935.00,947.00,946.00,946.00,946.00,946.00
mean,"595,100.50","309,635.36","1,215,384.93",11.00,25.24,7.62,7.52,3.11,"2,017.44","1,342,970.46","644,419.34","350,589.60"
std,"2,486,279.07","7,324,705.26","10,168,730.74",0.00,64.04,19.02,6.14,2.16,2.41,"11,440,024.69","2,699,466.96","8,220,571.36"
min,"-5,036,073.61","-23,515,438.00","-13,465,000.00",11.00,-94.00,0.00,0.00,1.00,"2,011.00","-13,465,011.47","-5,757,497.84","-26,549,117.94"
25%,-0.01,0.00,0.00,11.00,0.00,0.00,3.00,2.00,"2,015.00",0.00,-0.01,0.00
50%,0.00,0.00,"43,900.00",11.00,5.00,1.00,7.00,3.00,"2,017.00","48,569.09",0.00,0.00
75%,"299,762.50",0.00,"394,200.00",11.00,21.00,6.00,11.00,4.00,"2,020.00","433,296.93","329,713.96",0.00
max,"26,810,000.00","185,274,015.00","244,656,015.00",11.00,"1,098.00",255.00,41.00,16.00,"2,022.00","276,218,601.50","28,930,735.31","209,175,847.64"


In [117]:
transit.describe()

,fed_requested,ac_requested,total_requested,dist,dist_processing_days,hq_processing_days,fhwa_processing_days,seq,prepared_y,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
count,9.00,9.00,9.00,9.00,0.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00
mean,"3,288,888.89",0.00,"3,288,888.89",11.00,NaN,24.89,3.67,1.00,"2,020.44","3,258,552.53","3,258,552.53",0.00
std,"4,873,386.15",0.00,"4,873,386.15",0.00,NaN,6.17,1.32,0.00,0.88,"4,821,415.79","4,821,415.79",0.00
min,0.12,0.00,0.12,11.00,NaN,14.00,3.00,1.00,"2,020.00",0.13,0.13,0.00
25%,0.65,0.00,0.65,11.00,NaN,28.00,3.00,1.00,"2,020.00",0.68,0.68,0.00
50%,"12,354.63",0.00,"12,354.63",11.00,NaN,28.00,3.00,1.00,"2,020.00","12,935.05","12,935.05",0.00
75%,"9,342,958.12",0.00,"9,342,958.12",11.00,NaN,28.00,3.00,1.00,"2,020.00","9,637,984.26","9,637,984.26",0.00
max,"10,000,000.00",0.00,"10,000,000.00",11.00,NaN,28.00,6.00,1.00,"2,022.00","9,781,886.62","9,781,886.62",0.00


In [141]:
#pd.set_option("display.max_colwidth", 100)

In [142]:
(_dla_utils.calculate_data_all(df,
'total_requested', 'type_of_work',aggfunc="sum")>>arrange(-_.total_requested))


,type_of_work,total_requested
354,"To Replace The San Elijo Lagoon Bridge And Uc. Construct Hov Lanes, San Elijo","246,122,594.00"
30,Bridge Replacement (57c-0023),"138,469,728.00"
125,Hov Lane (tc),"124,051,798.00"
115,FTA Transfer,"119,339,975.00"
299,Replace 2-lane Bridge With 4-lane Bridge,"32,385,590.00"
...,...,...
282,Reconstruction Of Parking Lot & Passenger Loading,"3,260.71"
139,Install Curb Ramps And Sidewalk,"2,139.36"
52,Conduct Environmental Study For Railroad/ Local Street Grade Separation At,"1,545.64"
332,Safety Improvements,773.73


In [132]:
(_dla_utils.calculate_data_all((df>>filter(_.primary_agency_name=='Caltrans')),
'total_requested', 'type_of_work',aggfunc="mean")>>arrange(-_.total_requested))


,type_of_work,total_requested
16,To Replace The San Elijo Lagoon Bridge And Uc....,"123,061,297.00"
8,Hov Lane (tc),"41,350,599.33"
13,New Freeway Construction,"4,612,092.00"
18,"Widen Hwy From 2 To 4 Lanes,sr-98 & C.chavez (tc)","3,541,625.00"
3,Construct Noise Wall (tc),"1,747,500.00"
11,Lanscaping,"473,569.07"
6,Guardrail Var. Locations,"351,200.00"
10,Landscape Mitigation (tc),"250,000.00"
5,Construction Auxilary Lanes/ Widen Connector (tc),"220,000.00"
9,Install Curb Ramps And Sidewalk,"2,139.36"


### Top 50 Agencies with highest funds

In [143]:
((_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean"))>>arrange(-_.total_requested)).head(50)

,primary_agency_name,total_requested
3,Caltrans,"12,536,254.82"
16,Imperial County Transportation Commission,"3,656,763.71"
23,San Diego Association of Governments,"3,006,584.09"
22,San Diego,"2,548,911.28"
25,San Diego Metropolitan Transit System,"2,450,000.00"
27,San Marcos,"1,426,124.38"
1,Calexico,"1,040,094.85"
12,Escondido,"782,249.23"
5,Chula Vista,"637,313.89"
7,Coronado,"607,190.16"


In [122]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(31)
                        ), 'primary_agency_name','total_requested', 'primary_agency_name', subset,
                           'Average Total Requested Funds by Agency'
                       )

alt.Chart(...)

In [118]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(transit, 'total_requested', 'primary_agency_name', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(30)
                        ), 'primary_agency_name','total_requested', 'primary_agency_name', subset,
                           'Average Total Requested Funds by Agency: Transit Related Funding'
                       )

alt.Chart(...)

In [119]:
_dla_utils.basic_bar_chart((_dla_utils.calculate_data_all(transit_agencies, 'total_requested', aggregate_by='primary_agency_name', aggfunc="mean")),
                           'primary_agency_name', 'total_requested', 'primary_agency_name', subset,
                           'Transit Agencies Average Funding')



alt.Chart(...)

In [120]:
df>>filter(_.primary_agency_name=='San Diego Metropolitan Transit System')>>count(_.type_of_work)


,type_of_work,n
0,FTA Transfer,7
1,Low-floor Light Rail Transit Vehicles,1


In [121]:
df>>filter(_.primary_agency_name=='San Diego Metropolitan Transit System')>>count(_.prefix)

,prefix,n
0,FTASTPL,8


### Bottom 50 Agencies with lowest funds

In [64]:
avg_funds_bottom = (df>>group_by(_.primary_agency_name)>>summarize(avg_funds=_.total_requested.mean())>>arrange(-_.avg_funds)).tail(50)



In [65]:
avg_funds_bottom

,primary_agency_name,avg_funds
3,Caltrans,"12,536,254.82"
16,Imperial County Transportation Commission,"3,656,763.71"
23,San Diego Association of Governments,"3,006,584.09"
22,San Diego,"2,548,911.28"
25,San Diego Metropolitan Transit System,"2,450,000.00"
27,San Marcos,"1,426,124.38"
1,Calexico,"1,040,094.86"
12,Escondido,"782,249.23"
5,Chula Vista,"637,313.89"
7,Coronado,"607,190.16"


In [66]:
_dla_utils.basic_bar_chart((avg_funds_bottom.tail(40)), 'primary_agency_name','avg_funds', 'primary_agency_name', subset, 
                          'Lowest Average Total Funds by Agency')

WARN Symbol legend count exceeds limit, filtering items.


alt.Chart(...)

In [67]:
((
    _dla_utils.calculate_data_all(
        df, 'total_requested', 'primary_agency_name', aggfunc="mean"))
>>arrange(-_.total_requested)).tail(50)

,primary_agency_name,total_requested
3,Caltrans,"12,536,254.82"
16,Imperial County Transportation Commission,"3,656,763.71"
23,San Diego Association of Governments,"3,006,584.09"
22,San Diego,"2,548,911.28"
25,San Diego Metropolitan Transit System,"2,450,000.00"
27,San Marcos,"1,426,124.38"
1,Calexico,"1,040,094.85"
12,Escondido,"782,249.23"
5,Chula Vista,"637,313.89"
7,Coronado,"607,190.16"


In [68]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean"))
                          >>arrange(-_.total_requested)).tail(50)
                        ), 'primary_agency_name','total_requested', 'total_requested', subset,
                           'Average Total Requested Funds by Agency'
                       )

alt.Chart(...)

In [69]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
20009,Obligated,BRLS,5004(198),San Diego,2021-06-23,2021-08-27,2021-08-31,2021-09-03,2021-09-16,"103,580.00",0.00,"117,000.00",Authorized,5004,11.00,E-76 approved on,128.00,7.00,13.00,CAL44,Fairmount Avenue Overcrossing Aldine Drive (57...,Rehabilitate Bridge To Increase The Sufficienc...,2.00,2021-04-21,2021-08-27,SDAG,5004,"2,021.00",San Diego,"117,000.10","103,580.09",0.00


## Prefix Code Funding 

In [70]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'total_requested', 'prefix', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(30)
                        ), 'prefix','total_requested', 'prefix', subset,
                           'Average Total Requested Funds by Prefix'
                       )

alt.Chart(...)

In [71]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'fed_requested', 'prefix', aggfunc="mean"))
                          >>arrange(-_.fed_requested)).head(30)
                        ), 'prefix','fed_requested', 'prefix', subset,
                           'Average Federal Requested Funds by Prefix'
                       )

alt.Chart(...)

In [72]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'ac_requested', 'prefix', aggfunc="mean"))
                          >>arrange(-_.ac_requested)).head(30)
                        ), 'prefix','ac_requested', 'prefix', subset,
                           'Average Advanced Construction Requested Funds by Prefix'
                       )

alt.Chart(...)

## Functions to look by Prefix and by Agency: 

### By Agency:

In [73]:
_dla_utils.interactive_widget(df, 'primary_agency_name')

Dropdown(description='Primary_Agency_Name', options=('Brawley', 'Calexico', 'Calipatria', 'Caltrans', 'Carlsba…

Output()

### By Prefix:

In [74]:
_dla_utils.interactive_widget(df, 'prefix')

Dropdown(description='Prefix', options=('ACSTP', 'ATPL', 'ATPLNI', 'ATPLSB1', 'BHLO', 'BHLOZD', 'BHLS', 'BHLSC…

Output()